# Count top 200 persons via NER


We want to display top person names via NER.

NER實體命名辨識的結果有人名可以善加利用!


# Read data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('PTT_news_preprocessed.csv',sep='|')

In [3]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,Stock_04-16_1,04-16,學術股市,[新聞] 日媒：台積電將敲定面板級封裝規格 先從,原文標題：日媒：台積電將敲定面板級封裝規格 先從較小尺寸開始原文連結：發布時間：2025-0...,暫無,暫無,"[('台積電', 5), ('原文', 3), ('技術', 3), ('日媒', 2), ...","['原文', '標題', '：', '日媒', '：', '台積電', '將', '敲定',...","['原文', '標題', '日媒', '台積電', '面板級', '規格', '尺寸', '...","[NerToken(word='日媒', ner='NORP', idx=(5, 7)), ...","[('原文', 'Na'), ('標題', 'Na'), ('：', 'COLONCATEG...",https://www.ptt.cc/bbs/Stock/M.1744794518.A.3D...,NaN


In [4]:
# Take a look at the named entities of the first news 
df.entities[0]

"[NerToken(word='日媒', ner='NORP', idx=(5, 7)), NerToken(word='劉忠勇', ner='PERSON', idx=(61, 64)), NerToken(word='積電', ner='ORG', idx=(75, 77)), NerToken(word='2027年', ner='DATE', idx=(105, 110)), NerToken(word='台積', ner='ORG', idx=(121, 123)), NerToken(word='第一', ner='ORDINAL', idx=(130, 132)), NerToken(word='300x300', ner='CARDINAL', idx=(139, 146)), NerToken(word='台積電', ner='ORG', idx=(193, 196)), NerToken(word='桃園', ner='GPE', idx=(229, 231)), NerToken(word='2027年', ner='DATE', idx=(241, 246)), NerToken(word='四年', ner='DATE', idx=(267, 269)), NerToken(word='第一', ner='ORDINAL', idx=(308, 310)), NerToken(word='第二', ner='ORDINAL', idx=(316, 318))]"

In [5]:
# NerToken(word='烏克蘭', ner='GPE', idx=(4, 7))  # call function NerToken with three parameters: word, ner, and idx
def NerToken(word, ner, idx):
    # print(ner,word)
    return ner,word


In [6]:
# call function NerToken with three parameters: word, ner, and idx
# 回傳 關鍵字 與 NER
NerToken(word='烏克蘭', ner='GPE', idx=(4, 7))

('GPE', '烏克蘭')

In [7]:
# eval評估字串中的每個物件時，遇到NerToken(word='烏克蘭', ner='GPE', idx=(4, 7))，就會自動呼叫NerToken(word='烏克蘭', ner='GPE', idx=(4, 7))方法，而取得我們要的關鍵字、Ner
eval(df.entities[0])

[('NORP', '日媒'),
 ('PERSON', '劉忠勇'),
 ('ORG', '積電'),
 ('DATE', '2027年'),
 ('ORG', '台積'),
 ('ORDINAL', '第一'),
 ('CARDINAL', '300x300'),
 ('ORG', '台積電'),
 ('GPE', '桃園'),
 ('DATE', '2027年'),
 ('DATE', '四年'),
 ('ORDINAL', '第一'),
 ('ORDINAL', '第二')]

In [8]:
# We need the name and keyword
for ner,word in eval(df.entities[0]):
    print(ner,word)

NORP 日媒
PERSON 劉忠勇
ORG 積電
DATE 2027年
ORG 台積
ORDINAL 第一
CARDINAL 300x300
ORG 台積電
GPE 桃園
DATE 2027年
DATE 四年
ORDINAL 第一
ORDINAL 第二


In [9]:
from collections import Counter

In [10]:
# Filter condition: two words and specified NE
# 過濾條件:兩個字以上 特定的詞性
allowedNE=['PERSON']
#allowedNE=['PRODUCT']

In [11]:
filtered_words =[]
for ner,word in eval(df.entities[0]):
    if (len(word) >= 2) & (ner in allowedNE):
        filtered_words.append(word)

In [12]:
filtered_words

['劉忠勇']

In [13]:
counter = Counter( filtered_words )
counter.most_common( 200 )

[('劉忠勇', 1)]

# Define a function to count top 200 word frequency from entities for a piece of news

In [14]:
def ne_word_frequency( a_news_ne ):
    filtered_words =[]
    for ner,word in a_news_ne:
        if (len(word) >= 2) & (ner in allowedNE):
            filtered_words.append(word)
    counter = Counter( filtered_words )
    return counter.most_common( 200 )

In [15]:
ne_word_frequency(eval(df.entities[0]))

[('劉忠勇', 1)]

In [16]:
ne_word_frequency(eval(df.entities[1]))

[('川普', 1)]

In [17]:
ne_word_frequency(eval(df.entities[2]))

[('川普', 4), ('吳美依', 1), ('川普發文', 1), ('薩爾瓦多', 1), ('布格磊', 1), ('鮑爾', 1)]

## How to calculate frequency a couple of news?
### Concatenate (add or combine) two lists

In [18]:
# Let's define a empty list to store NEs of a news category
ne_group=[]

In [19]:
# concatenate the first NE list
ne_group+=eval(df.entities[0]) 

In [20]:
ne_group

[('NORP', '日媒'),
 ('PERSON', '劉忠勇'),
 ('ORG', '積電'),
 ('DATE', '2027年'),
 ('ORG', '台積'),
 ('ORDINAL', '第一'),
 ('CARDINAL', '300x300'),
 ('ORG', '台積電'),
 ('GPE', '桃園'),
 ('DATE', '2027年'),
 ('DATE', '四年'),
 ('ORDINAL', '第一'),
 ('ORDINAL', '第二')]

In [21]:
# concatenate the second NE list
ne_group += eval(df.entities[1]) 

In [22]:
ne_group

[('NORP', '日媒'),
 ('PERSON', '劉忠勇'),
 ('ORG', '積電'),
 ('DATE', '2027年'),
 ('ORG', '台積'),
 ('ORDINAL', '第一'),
 ('CARDINAL', '300x300'),
 ('ORG', '台積電'),
 ('GPE', '桃園'),
 ('DATE', '2027年'),
 ('DATE', '四年'),
 ('ORDINAL', '第一'),
 ('ORDINAL', '第二'),
 ('ORG', '美國鋼鐵'),
 ('ORDINAL', '第一'),
 ('GPE', '日本'),
 ('GPE', '美國'),
 ('GPE', '日本'),
 ('GPE', '美國'),
 ('GPE', '日本'),
 ('GPE', '日本'),
 ('GPE', '美國'),
 ('CARDINAL', '2018'),
 ('GPE', '中國'),
 ('GPE', '美國'),
 ('GPE', '中國'),
 ('GPE', '中國'),
 ('DATE', '幾年'),
 ('GPE', '中國'),
 ('DATE', '當年'),
 ('ORG', '政府'),
 ('ORG', '台積電會'),
 ('GPE', '中國'),
 ('GPE', '中國'),
 ('GPE', '美國'),
 ('GPE', '新加坡'),
 ('GPE', '台'),
 ('GPE', '中國'),
 ('GPE', '中國'),
 ('GPE', '中國'),
 ('ORG', '中國會'),
 ('GPE', '中國'),
 ('GPE', '美國'),
 ('GPE', '中國'),
 ('GPE', '中國'),
 ('GPE', '中國'),
 ('GPE', '美國'),
 ('GPE', '中國'),
 ('GPE', '日韓'),
 ('GPE', '中國'),
 ('GPE', '中國'),
 ('GPE', '中國'),
 ('GPE', '中國'),
 ('GPE', '斯里蘭卡'),
 ('FAC', '漢班托塔港'),
 ('GPE', '美國'),
 ('CARDINAL', '兩'),
 ('GPE', '中國'),
 ('GPE',

# Count top 200 word frequencies for each category

In [45]:
news_categories=['學術股市','綜合','閒談','棒球','軍事']

In [46]:

# The categorial word freqencies are keep in this dict. 
top_cate_ne_words={}

# Define a empty list to store all NEs in all categories "全部"
ne_all=[]

for category in news_categories:
    
    df_group = df[df.category == category]
    
    # Define a empty list to store NEs of a news category
    ne_group = []

    # concatenate all NEs in a category
    for ne in df_group.entities:
        ne_group += eval(ne)

    # concatenate all NEs for category "全部" (合併全部的詞不分類別)
    ne_all += ne_group

    # Calculate top words by calling ne_word_frequency() function
    topwords = ne_word_frequency( ne_group )
    
    # Keep topwords into dict
    top_cate_ne_words[category]=topwords

# Calculate top words for category '全部'
top_ne_all = ne_word_frequency(ne_all)
# Add topwors of category '全部' into dict
top_cate_ne_words['全部']=top_ne_all

In [47]:
top_cate_ne_words

{'學術股市': [('川普', 44),
  ('劉鏡清', 10),
  ('彭双浪', 8),
  ('林伯豐', 8),
  ('李成鋼', 5),
  ('貝森特', 4),
  ('黃仁勳', 4),
  ('凱衛', 3),
  ('川皇', 3),
  ('聯詠', 2),
  ('習寶', 2),
  ('林恩如', 2),
  ('劉庭宇', 2),
  ('袁詩羽', 2),
  ('王受文', 2),
  ('劉忠勇', 1),
  ('吳美依', 1),
  ('川普發文', 1),
  ('薩爾瓦多', 1),
  ('布格磊', 1),
  ('鮑爾', 1),
  ('胡錦濤', 1),
  ('瑞昱', 1),
  ('林瑩真', 1),
  ('王曉萍', 1),
  ('彭博', 1),
  ('吳欣恬', 1),
  ('郭智輝', 1),
  ('李慶華', 1),
  ('輝達', 1),
  ('川寶', 1),
  ('5. ', 1),
  ('佩珊', 1),
  ('張曉雯', 1),
  ('老川寶', 1),
  ('阿川', 1),
  ('chatgpt', 1),
  ('川普式', 1),
  ('范德賴恩', 1),
  ('施凱爾', 1),
  ('川爺', 1),
  ('張淑伶', 1),
  ('陳鎧妤', 1),
  ('川寶投', 1),
  ('張佩芬', 1),
  ('楊喻斐', 1),
  ('柯富仁', 1),
  ('彭双', 1),
  ('9萬多', 1),
  ('黃淑玲', 1),
  ('習近平', 1),
  ('貝森', 1),
  ('李維特', 1),
  ('陳清', 1),
  ('清楚川普', 1),
  ('李孟恩', 1),
  ('Fouquet', 1),
  ('徐筱嵐', 1),
  ('川普宣布', 1),
  ('白蘭氏', 1),
  ('桂格', 1),
  ('老協', 1),
  ('保守', 1)],
 '綜合': [('邱議瑩', 8),
  ('波特王', 8),
  ('朱立倫', 8),
  ('吐瓦魯', 5),
  (' ‧ ', 3),
  ('王阿平', 3),
  ('川普', 3),
  ('蔣萬安', 

In [48]:
top_cate_ne_words.items()

dict_items([('學術股市', [('川普', 44), ('劉鏡清', 10), ('彭双浪', 8), ('林伯豐', 8), ('李成鋼', 5), ('貝森特', 4), ('黃仁勳', 4), ('凱衛', 3), ('川皇', 3), ('聯詠', 2), ('習寶', 2), ('林恩如', 2), ('劉庭宇', 2), ('袁詩羽', 2), ('王受文', 2), ('劉忠勇', 1), ('吳美依', 1), ('川普發文', 1), ('薩爾瓦多', 1), ('布格磊', 1), ('鮑爾', 1), ('胡錦濤', 1), ('瑞昱', 1), ('林瑩真', 1), ('王曉萍', 1), ('彭博', 1), ('吳欣恬', 1), ('郭智輝', 1), ('李慶華', 1), ('輝達', 1), ('川寶', 1), ('5. ', 1), ('佩珊', 1), ('張曉雯', 1), ('老川寶', 1), ('阿川', 1), ('chatgpt', 1), ('川普式', 1), ('范德賴恩', 1), ('施凱爾', 1), ('川爺', 1), ('張淑伶', 1), ('陳鎧妤', 1), ('川寶投', 1), ('張佩芬', 1), ('楊喻斐', 1), ('柯富仁', 1), ('彭双', 1), ('9萬多', 1), ('黃淑玲', 1), ('習近平', 1), ('貝森', 1), ('李維特', 1), ('陳清', 1), ('清楚川普', 1), ('李孟恩', 1), ('Fouquet', 1), ('徐筱嵐', 1), ('川普宣布', 1), ('白蘭氏', 1), ('桂格', 1), ('老協', 1), ('保守', 1)]), ('綜合', [('邱議瑩', 8), ('波特王', 8), ('朱立倫', 8), ('吐瓦魯', 5), (' ‧ ', 3), ('王阿平', 3), ('川普', 3), ('蔣萬安', 3), ('柴柴', 3), ('宋建樑', 3), ('戴斐立', 3), ('李永得', 2), ('賴瑞隆', 2), ('羅瑞勳', 2), ('黃柏瀚', 2), ('陳麗娜', 2), ('王佩翊', 2), ('王浩宇', 2), ('

### To conveniently save data using pandas, we should convert dict to list.

In [49]:
list(top_cate_ne_words.items())

[('學術股市',
  [('川普', 44),
   ('劉鏡清', 10),
   ('彭双浪', 8),
   ('林伯豐', 8),
   ('李成鋼', 5),
   ('貝森特', 4),
   ('黃仁勳', 4),
   ('凱衛', 3),
   ('川皇', 3),
   ('聯詠', 2),
   ('習寶', 2),
   ('林恩如', 2),
   ('劉庭宇', 2),
   ('袁詩羽', 2),
   ('王受文', 2),
   ('劉忠勇', 1),
   ('吳美依', 1),
   ('川普發文', 1),
   ('薩爾瓦多', 1),
   ('布格磊', 1),
   ('鮑爾', 1),
   ('胡錦濤', 1),
   ('瑞昱', 1),
   ('林瑩真', 1),
   ('王曉萍', 1),
   ('彭博', 1),
   ('吳欣恬', 1),
   ('郭智輝', 1),
   ('李慶華', 1),
   ('輝達', 1),
   ('川寶', 1),
   ('5. ', 1),
   ('佩珊', 1),
   ('張曉雯', 1),
   ('老川寶', 1),
   ('阿川', 1),
   ('chatgpt', 1),
   ('川普式', 1),
   ('范德賴恩', 1),
   ('施凱爾', 1),
   ('川爺', 1),
   ('張淑伶', 1),
   ('陳鎧妤', 1),
   ('川寶投', 1),
   ('張佩芬', 1),
   ('楊喻斐', 1),
   ('柯富仁', 1),
   ('彭双', 1),
   ('9萬多', 1),
   ('黃淑玲', 1),
   ('習近平', 1),
   ('貝森', 1),
   ('李維特', 1),
   ('陳清', 1),
   ('清楚川普', 1),
   ('李孟恩', 1),
   ('Fouquet', 1),
   ('徐筱嵐', 1),
   ('川普宣布', 1),
   ('白蘭氏', 1),
   ('桂格', 1),
   ('老協', 1),
   ('保守', 1)]),
 ('綜合',
  [('邱議瑩', 8),
   ('波特王', 8),
   ('朱立倫'

# All-in-one function  for calculating the most popular persons

In [50]:
allowedNE=['PERSON']

In [51]:
news_categories=['學術股市','綜合','閒談','棒球','軍事']

In [52]:
def ne_word_frequency( a_news_ne ):
    filtered_words =[]
    for ner,word in a_news_ne:
        if (len(word) >= 2) & (ner in allowedNE):
            filtered_words.append(word)
    counter = Counter( filtered_words )
    return counter.most_common( 200 )

In [53]:
# All-in-one function
def get_top_ner_words():
    top_cate_ner_words={}
    words_all=[]
    for category in news_categories:
        df_group = df[df.category == category]
        words_group = []

        # concatenate terms in a category
        for row in df_group.entities:
            words_group += eval(row)

        # concatenate all terms
        words_all += words_group

        # Get top words by calling ne_word_frequency() function
        topwords = ne_word_frequency( words_group )
        top_cate_ner_words[category] = topwords

    topwords_all = ne_word_frequency(words_all)
    top_cate_ner_words['全部'] = topwords_all
    
    return list(top_cate_ner_words.items())
    #return top_cate_ne_words

In [54]:
%%time
hotPersons = get_top_ner_words()

CPU times: total: 78.1 ms
Wall time: 91.5 ms


In [55]:
hotPersons

[('學術股市',
  [('川普', 44),
   ('劉鏡清', 10),
   ('彭双浪', 8),
   ('林伯豐', 8),
   ('李成鋼', 5),
   ('貝森特', 4),
   ('黃仁勳', 4),
   ('凱衛', 3),
   ('川皇', 3),
   ('聯詠', 2),
   ('習寶', 2),
   ('林恩如', 2),
   ('劉庭宇', 2),
   ('袁詩羽', 2),
   ('王受文', 2),
   ('劉忠勇', 1),
   ('吳美依', 1),
   ('川普發文', 1),
   ('薩爾瓦多', 1),
   ('布格磊', 1),
   ('鮑爾', 1),
   ('胡錦濤', 1),
   ('瑞昱', 1),
   ('林瑩真', 1),
   ('王曉萍', 1),
   ('彭博', 1),
   ('吳欣恬', 1),
   ('郭智輝', 1),
   ('李慶華', 1),
   ('輝達', 1),
   ('川寶', 1),
   ('5. ', 1),
   ('佩珊', 1),
   ('張曉雯', 1),
   ('老川寶', 1),
   ('阿川', 1),
   ('chatgpt', 1),
   ('川普式', 1),
   ('范德賴恩', 1),
   ('施凱爾', 1),
   ('川爺', 1),
   ('張淑伶', 1),
   ('陳鎧妤', 1),
   ('川寶投', 1),
   ('張佩芬', 1),
   ('楊喻斐', 1),
   ('柯富仁', 1),
   ('彭双', 1),
   ('9萬多', 1),
   ('黃淑玲', 1),
   ('習近平', 1),
   ('貝森', 1),
   ('李維特', 1),
   ('陳清', 1),
   ('清楚川普', 1),
   ('李孟恩', 1),
   ('Fouquet', 1),
   ('徐筱嵐', 1),
   ('川普宣布', 1),
   ('白蘭氏', 1),
   ('桂格', 1),
   ('老協', 1),
   ('保守', 1)]),
 ('綜合',
  [('邱議瑩', 8),
   ('波特王', 8),
   ('朱立倫'

In [56]:
## Save top-200 hot persons for future usage

In [57]:
df_hotPersons = pd.DataFrame(hotPersons, columns = ['category','top_keys'])

In [58]:
df_hotPersons

,category,top_keys
0,學術股市,"[(川普, 44), (劉鏡清, 10), (彭双浪, 8), (林伯豐, 8), (李成鋼..."
1,綜合,"[(邱議瑩, 8), (波特王, 8), (朱立倫, 8), (吐瓦魯, 5), ( ‧ ,..."
2,閒談,"[(青山, 6), (長谷川, 4), (谷川, 3), (阿音, 3), (柯南, 3),..."
3,棒球,"[(徐若熙, 21), (呂詠臻, 16), (蔣少宏, 14), (葉君璋, 10), (..."
4,軍事,"[(川普, 56), (顧立雄, 13), (澤倫斯基, 13), (萬斯, 7), (普丁..."
5,全部,"[(川普, 103), (徐若熙, 21), (呂詠臻, 16), (蔣少宏, 14), (..."


## Save the word frequecy 

In [67]:
df_hotPersons.to_csv('PTTnews_top_person_by_category_via_ner.csv', sep=',', index=False)

# For your reference: Another way to analyze by concatenating counter objects


In [60]:
from collections import Counter
a = Counter({'menu': 20, 'good': 15, 'happy': 10, 'bar': 5})
b = Counter({'menu': 1, 'good': 1, 'bar': 3})
a + b

Counter({'menu': 21, 'good': 16, 'happy': 10, 'bar': 8})

In [61]:
df.entities[0]

"[NerToken(word='日媒', ner='NORP', idx=(5, 7)), NerToken(word='劉忠勇', ner='PERSON', idx=(61, 64)), NerToken(word='積電', ner='ORG', idx=(75, 77)), NerToken(word='2027年', ner='DATE', idx=(105, 110)), NerToken(word='台積', ner='ORG', idx=(121, 123)), NerToken(word='第一', ner='ORDINAL', idx=(130, 132)), NerToken(word='300x300', ner='CARDINAL', idx=(139, 146)), NerToken(word='台積電', ner='ORG', idx=(193, 196)), NerToken(word='桃園', ner='GPE', idx=(229, 231)), NerToken(word='2027年', ner='DATE', idx=(241, 246)), NerToken(word='四年', ner='DATE', idx=(267, 269)), NerToken(word='第一', ner='ORDINAL', idx=(308, 310)), NerToken(word='第二', ner='ORDINAL', idx=(316, 318))]"

In [62]:
# conver string to set
eval(df.entities[0])

[('NORP', '日媒'),
 ('PERSON', '劉忠勇'),
 ('ORG', '積電'),
 ('DATE', '2027年'),
 ('ORG', '台積'),
 ('ORDINAL', '第一'),
 ('CARDINAL', '300x300'),
 ('ORG', '台積電'),
 ('GPE', '桃園'),
 ('DATE', '2027年'),
 ('DATE', '四年'),
 ('ORDINAL', '第一'),
 ('ORDINAL', '第二')]

In [63]:
top_cate_ner_words={} # final result
counter_all = Counter() # counter for category '全部'
for category in news_categories:
    
    df_group = df[df.category == category]
    
    # concatenate all filtered words in the same category
    words_group = []
    for row in df_group.entities:
        
        # filter words for each news
        filtered_words =[]
        for (ner, word) in eval(row):
            if (len(word) >= 2) & (ner in allowedNE):
                filtered_words.append(word)
                
        # concatenate filtered words  
        words_group += filtered_words

    # now we can count word frequency
    counter = Counter( words_group )
    
    # combine counter_all with conter
    counter_all += counter

    # get a few most common items
    topwords = counter.most_common(200)

    # store topwords
    top_cate_ner_words[category]= topwords

# Process category '全部'
top_cate_ne_words['全部'] = counter_all.most_common(200)

In [64]:
# All-in-one function
def get_top_ner_words():
    
    top_cate_ner_words={} # final result
    counter_all = Counter() # counter for category '全部'
    for category in news_categories:

        df_group = df[df.category == category]

        # concatenate all filtered words in the same category
        words_group = []
        for row in df_group.entities:

            # filter words for each news
            filtered_words =[]
            for (ner, word) in eval(row):
                if (len(word) >= 2) & (ner in allowedNE):
                    filtered_words.append(word)

            # concatenate filtered words  
            words_group += filtered_words

        # now we can count word frequency
        counter = Counter( words_group )

        # combine counter_all with conter
        counter_all += counter
        
        # get a few most common items
        topwords = counter.most_common(200)

        # store topwords
        top_cate_ner_words[category]= topwords

    # Process category '全部'
    top_cate_ner_words['全部'] = counter_all.most_common(200)
    return list(top_cate_ner_words.items())

In [65]:
%%time
result = get_top_ner_words()

CPU times: total: 78.1 ms
Wall time: 73.5 ms


In [66]:
result

[('學術股市',
  [('川普', 44),
   ('劉鏡清', 10),
   ('彭双浪', 8),
   ('林伯豐', 8),
   ('李成鋼', 5),
   ('貝森特', 4),
   ('黃仁勳', 4),
   ('凱衛', 3),
   ('川皇', 3),
   ('聯詠', 2),
   ('習寶', 2),
   ('林恩如', 2),
   ('劉庭宇', 2),
   ('袁詩羽', 2),
   ('王受文', 2),
   ('劉忠勇', 1),
   ('吳美依', 1),
   ('川普發文', 1),
   ('薩爾瓦多', 1),
   ('布格磊', 1),
   ('鮑爾', 1),
   ('胡錦濤', 1),
   ('瑞昱', 1),
   ('林瑩真', 1),
   ('王曉萍', 1),
   ('彭博', 1),
   ('吳欣恬', 1),
   ('郭智輝', 1),
   ('李慶華', 1),
   ('輝達', 1),
   ('川寶', 1),
   ('5. ', 1),
   ('佩珊', 1),
   ('張曉雯', 1),
   ('老川寶', 1),
   ('阿川', 1),
   ('chatgpt', 1),
   ('川普式', 1),
   ('范德賴恩', 1),
   ('施凱爾', 1),
   ('川爺', 1),
   ('張淑伶', 1),
   ('陳鎧妤', 1),
   ('川寶投', 1),
   ('張佩芬', 1),
   ('楊喻斐', 1),
   ('柯富仁', 1),
   ('彭双', 1),
   ('9萬多', 1),
   ('黃淑玲', 1),
   ('習近平', 1),
   ('貝森', 1),
   ('李維特', 1),
   ('陳清', 1),
   ('清楚川普', 1),
   ('李孟恩', 1),
   ('Fouquet', 1),
   ('徐筱嵐', 1),
   ('川普宣布', 1),
   ('白蘭氏', 1),
   ('桂格', 1),
   ('老協', 1),
   ('保守', 1)]),
 ('綜合',
  [('邱議瑩', 8),
   ('波特王', 8),
   ('朱立倫'